In [1]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\John\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [3]:
# URL of page to be scraped
url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
browser.visit(url)

In [16]:
addresses = []
properties_info = []

pages = np.arange(0, 3, 1)

for page in pages:
    
    page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

[['Sold $516,000 in Jul 2021',
  'Apartment: ',
  'Land size: 157 sqm',
  'Agent: David Airey',
  'Street view | Nearby'],
 ['Sold $325,000 in Jul 2021', 'Apartment: ', 'Street view | Nearby'],
 ['Sold $373,000 in Jul 2021',
  'Last Sold $361,000 in Mar 2009',
  'Apartment: 2  1  1 ',
  'Land size: 2,495 sqm | Building size: 100 sqm',
  'Agent: Sam Dempsey',
  'Spacious 2 bedroom top floor apartment in beautiful condition with many extras and secure parking. More information to follow. Register your interest today. Phone Ron Goddard 0449 757 945.more',
  'Street view | Nearby'],
 ['Sold $450,000 in Jul 2021',
  'Rent $465pw in Nov 2016',
  'Apartment: 2  2  1 ',
  'Land size: 109 sqm | Building size: 70 sqm',
  'Agent: Danielle Geagea',
  'Street view | Nearby'],
 ['Sold $802,000 in Jul 2021',
  'Last Sold $735,000 in Sep 2013',
  'Rent $590pw in Oct 2018',
  'Townhouse: 3  1  2 ',
  'Land size: 104 sqm',
  "Agent: Annette O'Brien-Oxley",
  'Lot/Plan No: 2/S009756more',
  'Floorplan | 

In [ ]:
tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

In [17]:
addresses

['39/128 Mounts Bay Road',
 '32/418 Murray Street',
 '40/120 Lake Street',
 '11/177 Stirling Street',
 '21 Myrtle Street',
 '5/19 Lindsay Street',
 '8/124 Mounts Bay Road',
 '26/14 Money Street',
 '104/305 Murray Street',
 '401/237 Adelaide Terrace',
 '45/34 Palmerston Street',
 '2009/11 Barrack Square',
 '25/226 Beaufort Street',
 '30/69 Milligan Street',
 '11/18 Robinson Avenue',
 '56/378 Beaufort Street',
 '172 Lake Street',
 '109/15 Aberdeen Street',
 '17/34 Palmerston Street',
 '47/76 Newcastle Street',
 '17/34 Bulwer Street',
 '1/478 William Street',
 '2701/237 Adelaide Terrace',
 '601/112 Mounts Bay Road',
 '73/262 Lord Street',
 '48B Randell Street',
 '36/10 Pendal Lane',
 '10 Pisconeri Street',
 '2801/237 Adelaide Terrace',
 '392 Beaufort Street']

In [ ]:
page_info = []
for i in range(2,12):
    try:
        info = []
        trs = tables[i].find_all('tr')
        for tr in trs:
            info.append(tr.td.text)
    except:
        info.append('')
    page_info.append(info)
page_info

In [18]:
Properties = []
for i in range(0, len(properties_info)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '39/128 Mounts Bay Road',
  'price': '$516,000',
  'sold date': 'Jul 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'Land size': '157',
  'agent': 'David Airey'},
 {'address': '32/418 Murray Street',
  'price': '$325,000',
  'sold date': 'Jul 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': ''},
 {'address': '40/120 Lake Street',
  'price': '$373,000',
  'sold date': 'Jul 2021',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '1',
  'car space': '1',
  'Land size': '2,495',
  'Building size': '100',
  'agent': 'Sam Dempsey'},
 {'address': '11/177 Stirling Street',
  'price': '$450,000',
  'sold date': 'Jul 2021',
  'rent': '$465pw',
  'rent date': 'Nov 2016',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '2',
  'car space': '1',
  'Land size': '109',
  'Building size': '70',
  'agent': 'Danielle Geagea'},
 {'address': '21 Myrtle Street',
  'price': '$802,000',
  'sold date': 'Jul 

In [19]:
len(Properties)

30